In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
import sys
sys.path.append('/home/nam4/Desktop/sandbox/')
import ml_inspector

In [ ]:
# TODO: Optimize pipelines below using GridSearchCV with MY CODE (opt-sklearn-pipe)

# Classification

In [18]:
head = 'classification/vanilla/3/'
m = pickle.load(open('{}/fitted_model.pkl'.format(head), 'rb'))
x_test = pickle.load(open('{}/X_test.pkl'.format(head), 'rb'))
y_test = pickle.load(open('{}/y_test.pkl'.format(head), 'rb'))
x_train = pickle.load(open('{}/X_train.pkl'.format(head), 'rb'))
y_train = pickle.load(open('{}/y_train.pkl'.format(head), 'rb'))

In [19]:
m.score(x_test, y_test) # not very good

0.7647058823529411

In [20]:
m.score(x_train, y_train) # surprisingly good

0.984375

In [22]:
cv_res = pd.DataFrame(m.cv_results_)

In [108]:
np.unique(cv_res.sort_values(['mean_test_score'], ascending=False)['param_classifier:__choice__'].head(13000), return_counts=True)
# most of models that are best are passive aggressive

(array(['MLPClassifier', 'bernoulli_nb', 'extra_trees', 'liblinear_svc',
        'passive_aggressive', 'random_forest', 'sgd'], dtype=object),
 array([    2,     1,     1,     1, 12974,     2,    19]))

In [109]:
all_models = cv_res['param_classifier:__choice__'].unique()

In [110]:
top = None
for model in all_models:
    x = cv_res[cv_res['param_classifier:__choice__'] == model].sort_values(['mean_test_score'], ascending=False).head(1)
    if top is None:
        top = x
    else:
        top = pd.concat((top, x), ignore_index=True)

In [111]:
top.sort_values(['mean_test_score'], ascending=False)

,mean_test_score,mean_fit_time,params,rank_test_scores,status,budgets,param_balancing:strategy,param_classifier:__choice__,param_data_preprocessing:categorical_transformer:categorical_encoding:__choice__,param_data_preprocessing:categorical_transformer:category_coalescence:__choice__,...,param_feature_preprocessor:RFE:svc_loss,param_feature_preprocessor:RFE:svc_penalty,param_feature_preprocessor:fast_ica:n_components,param_feature_preprocessor:kernel_pca:coef0,param_feature_preprocessor:kernel_pca:degree,param_feature_preprocessor:kernel_pca:gamma,param_feature_preprocessor:nystroem_sampler:coef0,param_feature_preprocessor:nystroem_sampler:degree,param_feature_preprocessor:nystroem_sampler:gamma,param_feature_preprocessor:RFE:lr_intercept_scaling
2,0.968750,8.256669,"{'balancing:strategy': 'weighting', 'classifie...",1,Success,0.0,weighting,passive_aggressive,one_hot_encoding,minority_coalescer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.937500,7.337642,"{'balancing:strategy': 'none', 'classifier:__c...",12306,Success,0.0,none,sgd,no_encoding,minority_coalescer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.921875,26.538084,"{'balancing:strategy': 'none', 'classifier:__c...",12543,Success,0.0,none,random_forest,one_hot_encoding,no_coalescense,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.921875,40.495477,"{'balancing:strategy': 'weighting', 'classifie...",12543,Success,0.0,weighting,lda,no_encoding,no_coalescense,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.921875,26.211239,"{'balancing:strategy': 'weighting', 'classifie...",12543,Success,0.0,weighting,extra_trees,no_encoding,no_coalescense,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.921875,20.974545,"{'balancing:strategy': 'weighting', 'classifie...",12543,Success,0.0,weighting,k_nearest_neighbors,no_encoding,minority_coalescer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.921875,21.159492,"{'balancing:strategy': 'weighting', 'classifie...",12543,Success,0.0,weighting,bernoulli_nb,no_encoding,minority_coalescer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.921875,16.839201,"{'balancing:strategy': 'weighting', 'classifie...",12543,Success,0.0,weighting,libsvm_svc,no_encoding,no_coalescense,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,0.921875,79.389456,"{'balancing:strategy': 'weighting', 'classifie...",12543,Success,0.0,weighting,MLPClassifier,one_hot_encoding,minority_coalescer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0.921875,20.875286,"{'balancing:strategy': 'none', 'classifier:__c...",12543,Success,0.0,none,liblinear_svc,no_encoding,minority_coalescer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
# Create some pipelines to optimize eventually
from sklearn.pipeline import Pipeline
import sklearn

In [212]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier as ERT
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import PassiveAggressiveClassifier
pipeline1 = Pipeline([('Scaler', StandardScaler()),
                      ('ERT', SelectFromModel(ERT(bootstrap=False,
                                                  criterion='gini',
                                                  max_features=0.4394463397373907,
                                                  n_estimators=100,
                                                  random_state=0
                                                ),
                                             threshold='mean'
                                            )),
                      ('PAC', PassiveAggressiveClassifier(C=0.03575548812589056,
                                                          fit_intercept=True,
                                                          loss='hinge',
                                                          average=False,
                                                          random_state=0,
                                                          class_weight='balanced'
                                                         ))
                     ])

from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
pipeline2 = Pipeline([('Scaler', StandardScaler()), 
                     ('LDA', LDA(solver='svd',
                                 n_components=9, # only 2 classes though so will go to 1 component
                                )),
                     ('RF', RandomForestClassifier(criterion='gini',
                                                   bootstrap=True,
                                                   max_depth=None,
                                                   max_features=0.12324407123727832,
                                                   class_weight='balanced'
                                                  ))]) 


from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFwe
from sklearn.svm import SVC
pipeline3 = Pipeline([('Scaler', StandardScaler()), 
                     ('SelectFwe', SelectFwe(score_func=sklearn.feature_selection.f_classif,
                                             alpha=0.19252136279817358,
                                            )),
                     ('SVC', SVC(C=1310.8079513160242,
                                 gamma=3.759385850940499e-05,
                                 kernel='rbf',
                                 shrinking=True,
                                 class_weight='balanced'
                                ))]) 

# Bernoulli looks for discrete data, so binarize threshold will be important to optimize
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import BernoulliNB
pipeline4 = Pipeline([('Scaler', StandardScaler()), 
                     ('LDA', LDA(n_components=9, #again will just go to 1
                                 solver='svd'
                                )),
                     ('BernoulliNB', BernoulliNB(alpha=58.83897387376973,
                                                 fit_prior=False
                                ))]) 


from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier as ERT
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
pipeline5 = Pipeline([('Scaler', StandardScaler()),
                      ('ERT', SelectFromModel(ERT(bootstrap=False,
                                                 criterion='gini',
                                                 max_features=0.581689983760726,
                                                 n_estimators=100,
                                                  random_state=0
                                                ),
                                             threshold='mean'
                                            )),
                      ('LDA', LDA(n_components=22, # again will go to 1,
                                )
                     )])

In [213]:
def train(pipeline):
    def get_X():
        df = pd.read_csv('./data_Fish_Mucus_BV_200115_actual data.csv')
        X = df[df.Stock.isin(['CULL', 'BROOD'])]
        X = X.drop(['ID', 'Class', 'Strain', 'Fork Length (cm)', 'Weight (kg)'], axis=1)
        return X

    # Import Data
    X = get_X().drop(['Breeding Value', 'Stock'], axis=1).values
    y = get_X()['Stock'].values

    # Encode labels
    from sklearn.preprocessing import LabelEncoder
    enc = LabelEncoder()
    y = enc.fit_transform(y)

    # Stratified test/train split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                        random_state=0, 
                                                        shuffle=True, 
                                                        stratify=y)

    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import StratifiedKFold
    cvs = cross_val_score(pipeline, cv=StratifiedKFold(n_splits=10, # go up to 10
                                                       shuffle=True, 
                                                       random_state=0),
                          X=X_train,
                          y=y_train,
                          n_jobs=-1)

    
    print(['%.3f'%s for s in cvs])
    print('Training CV: ', np.mean(cvs), '+/-', np.std(cvs))
    f = pipeline.fit(X_train, y_train)
    print('Test Set: ', f.score(X_test, y_test))

In [214]:
# Without any further optimization, what are these performances?
# Looks like they are all about 90% for training CV, with tests >80%
# However, note that the standard deviation for these is very large so probably need more data
# before we can expect very good results.

# Folds of the CV look bimodal in performance

In [217]:
train(pipeline1)

['0.857', '1.000', '0.857', '1.000', '0.833', '1.000', '0.833', '0.833', '0.833', '0.833']
Training CV:  0.8880952380952382 +/- 0.0738095238095238
Test Set:  0.8235294117647058


In [220]:
train(pipeline2)

['0.857', '1.000', '0.857', '0.714', '0.833', '0.833', '0.667', '0.667', '1.000', '0.667']
Training CV:  0.8095238095238095 +/- 0.12187127516256885
Test Set:  0.7647058823529411


/home/nam4/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(260, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/nam4/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [223]:
train(pipeline3) # 5/10 folds are 100% on validation set

['0.857', '1.000', '0.714', '1.000', '1.000', '1.000', '0.833', '1.000', '0.833', '0.667']
Training CV:  0.8904761904761905 +/- 0.12196427118919712
Test Set:  0.7647058823529411


In [224]:
train(pipeline4) # again 5/10

['0.857', '1.000', '1.000', '0.857', '1.000', '1.000', '0.667', '0.833', '1.000', '0.667']
Training CV:  0.8880952380952382 +/- 0.12868161063687295
Test Set:  0.8235294117647058


/home/nam4/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(260, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/nam4/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [226]:
train(pipeline5)

['1.000', '0.429', '0.571', '0.857', '0.833', '0.833', '0.167', '0.500', '0.833', '0.667']
Training CV:  0.669047619047619 +/- 0.23962603139190644
Test Set:  0.6470588235294118


/home/nam4/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(66, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/nam4/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [ ]:
# BernoulliNB after scaling is an interesting result
# Performs similar to SVC

# Regression

In [ ]:
# Regression (as expected) is quite bad - best is worse than random/mean guessing on test
# but about random/mean on training

In [186]:
head = 'regression/vanilla/3/'
m = pickle.load(open('{}/fitted_model.pkl'.format(head), 'rb'))
x_test = pickle.load(open('{}/X_test.pkl'.format(head), 'rb'))
y_test = pickle.load(open('{}/y_test.pkl'.format(head), 'rb'))
x_train = pickle.load(open('{}/X_train.pkl'.format(head), 'rb'))
y_train = pickle.load(open('{}/y_train.pkl'.format(head), 'rb'))

In [187]:
m.score(x_test, y_test)

-0.1504223341520614

In [188]:
m.score(x_train, y_train)

1.0

In [189]:
print(m.sprint_statistics())

auto-sklearn results:
  Dataset name: Salmon Mucus v0
  Metric: r2
  Best validation score: -0.009021
  Number of target algorithm runs: 12991
  Number of successful target algorithm runs: 4891
  Number of crashed target algorithm runs: 3081
  Number of target algorithms that exceeded the time limit: 4552
  Number of target algorithms that exceeded the memory limit: 467

